We want to know what is the acceptable level of error on c_z for SDSS and LSST_DESI such that statistical error dominates the error budget.

In [1]:
%matplotlib inline
import numpy as np; import constrain_IA_BlazekMethod as Bl
import shared_functions_setup as setup
import matplotlib.pyplot as plt

SURVEY= SDSS


$\Delta \Sigma = c_z \widetilde{\Delta \Sigma}$. Therefore, $\frac{\sigma^2(\Delta \Sigma)}{\Delta \Sigma^2} = \sqrt{\frac{\sigma^2(c_z)}{c_z^2} + \frac{\sigma^2(\widetilde{\Delta\Sigma})}{\widetilde{\Delta\Sigma}^2}}$
Let us suppose that we want the error on $\Delta\Sigma$ due to $c_z$ to make up no more than $x\%$ of $\sigma(\Delta\Sigma)$.

In [ ]:
x=0.2 # 20 percent

We want to know for what values of $\sigma^2(c_z)$ is it true that $x^2 \ge \frac{\widetilde{\Delta\Sigma^2}\sigma^2(c_z)}{\widetilde{\Delta\Sigma}^2 \sigma^2(c_z) + c_z^2 \sigma^2(\widetilde{\Delta\Sigma})}$. This is equivalent to wanting to know when it is true that $\frac{\sigma(c_z)}{c_z} \le \frac{\sigma(\widetilde{\Delta \Sigma})}{\widetilde{\Delta \Sigma}} \frac{x}{\sqrt{1-x^2}}$. Let's assume that the covariance of the 'B' source sample galaxy galaxy lensing is characteristic of a typical lensing measurement we would want to make. It would probably be better to use integrated signal to noise to avoid considering only the diagonal, but this will do for the moment to get a rough estimate.

In [ ]:
#Get the statistical covariance matrix and statistical only signal to noise

SURVEY = 'SDSS'
if (SURVEY=='SDSS'):
	import params as pa
elif (SURVEY=='LSST_DESI'):
	import params_LSST_DESI as pa
else:
	print "We don't have support for that survey yet; exiting."
	exit()

cov_DS_stat_SDSS = np.loadtxt('./txtfiles/covmats/cov_DelSig_zLext_SDSS_sample=B_rpts2000_lpts100000_deltaz=0.17_HODupdate.txt')

# Get the theoretical lensing Delta Sigma 

rp_bins 	= 	setup.setup_rp_bins(pa.rp_min, pa.rp_max, pa.N_bins)
rp_cent		=	setup.rp_bins_mid(rp_bins)
DeltaSig = Bl.get_DeltaSig_theory(rp_bins, rp_cent)
# Load the fiducial cz and get the estimated lensing Delta Sigma
(cz_a_fid, cz_b_fid) = np.loadtxt('./txtfiles/cz/cz_afid_bfid_extl_survey=SDSS_deltaz=0.17_HODupdate.txt', unpack=True)
EstDeltaSig = np.asarray(DeltaSig) / cz_b_fid 

# Statistical only signal-to-noise
stat_StoNsq = np.dot(DeltaSig, np.dot(np.linalg.inv(cov_DS_stat_SDSS), DeltaSig))
print "StoN=", np.sqrt(stat_StoNsq)


In [ ]:
# Now make the systematics covariance matrix at a variety of values of fractional error on cz
frac_err_sys = np.zeros(len(pa.fudge_frac_level))
for fi in range(0, len(pa.fudge_frac_level)):
    cov_DS_sys_SDSS = np.zeros((len(rp_cent), len(rp_cent)))
    for rpi in range(0,len(rp_cent)):
        for rpj in range(0,len(rp_cent)):
            cov_DS_sys_SDSS = pa.fudge_frac_level[fi]**2 * DeltaSig[rpi] * DeltaSig[rpj]
            
    cov_tot = cov_DS_sys_SDSS + cov_DS_stat_SDSS
    StoN_tot_sq = np.dot(DeltaSig, np.dot(np.linalg.inv(cov_tot), DeltaSig))
    NtoS_sys = np.sqrt( StoN_tot_sq**(-1) - stat_StoNsq**(-1))
    frac_err_sys[fi] = NtoS_sys / (np.sqrt(StoN_tot_sq)**(-1))

In [ ]:
plt.figure()
plt.loglog(pa.fudge_frac_level, frac_err_sys)
plt.hold(True)
plt.axhline(0.2)
plt.show()

In [ ]:
# get the maximum allowed values of fractional error on cz
max_frac_err = np.diag(cov_DS_SDSS) / EstDeltaSig * x / (np.sqrt(1.-x**2))
print "The maximum fractional error allowed on cz in SDSS for max contribution to total error =", x,"fraction is", max_frac_err
